In [1]:
# https://python.langchain.com/v0.2/docs/tutorials/agents/

### Agent

- Agents\
  Agents are systems that use LLMs as reasoning engines to determine which actions to take and the inputs to pass them.


In [6]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults


search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)

tools = [search]

[{'url': 'https://world-weather.info/forecast/usa/san_francisco/july-2024/', 'content': 'Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ San Francisco Weather Forecast for July 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.'}, {'url': 'https://www.accuweather.com/en/us/san-francisco/94103/july-weather/347629', 'content': 'Get the monthly weather forecast for San Francisco, CA, including daily high/low, historical averages, to help you plan ahead.'}]


In [5]:
from langchain_core.messages import HumanMessage

from core.llm import CHAT_LLM


model_with_tools = CHAT_LLM.bind_tools(tools)


response = model_with_tools.invoke([HumanMessage("Hi!")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


# This isn't calling that tool yet - it's just telling us to. In order to actually call it, we'll want to create our agent.
response = model_with_tools.invoke([HumanMessage("What's the weather in SF?")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []
ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_T2qPZpnKFzlx4lDaayEcHyEZ', 'type': 'tool_call'}]


### Create and run the agent


In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(CHAT_LLM, tools)

In [9]:
response = agent_executor.invoke({"messages": [HumanMessage("hi!")]})
response["messages"]

[HumanMessage(content='hi!', id='0c88a436-3c90-4115-957d-e4274fddc300'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-80cd8e3a-4091-44aa-99ae-16d260074493-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [10]:
response = agent_executor.invoke(
    {"messages": [HumanMessage("what's the weather in SF?")]}
)
response["messages"]

[HumanMessage(content="what's the weather in SF?", id='b498c923-46b6-4d54-94e4-3d3d42d761f7'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6iEScCeiPfO49NeVvVzOepfE', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-d33a1b3b-0c06-44e7-84db-f641e6700e1d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_6iEScCeiPfO49NeVvVzOepfE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 21, 'total_tokens': 109}),
 ToolMessage(content='[{"url": "https://weatherspark.com/h/y/557/2024/Historical-Weather-during-2024-in-San-Francisco-California-United-Sta

### Streaming messages


In [11]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage("what's the weather in SF?")]}
):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QKj2aaVqqZFXThwqIYlTy7Px', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'tool_calls', 'logprobs': None, 'content_filter_results': {}}, id='run-1d4b4f5f-1ed1-4602-bc02-92c32e0bf710-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_QKj2aaVqqZFXThwqIYlTy7Px', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 21, 'total_tokens': 109})]}}
---
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.wunderground.com/hourly/us/ca/san-francisco/94120/date/2024-7-14", "content": "San Francisco Weather Forecasts. Weather Underground provid

### Streaming tokens

- This .astream_events method only works with Python 3.11 or higher.


In [15]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage("What's the weather in SF?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://world-weather.info/forecast/usa/san_francisco/july-2024/', 'content': 'Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ San Francisco Weather Forecast for July 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1720957422, 'localtime': '2024-07-14 4:43'}, 'current': {'last_updated_epoch': 1720956600, 'last_updated': '2024-07-14 04:30', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 9.4, 'wind_kph': 

### Adding in memory

- Use `thread_id` key (not `session_id`)


In [17]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = create_react_agent(CHAT_LLM, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage("Hi I'm Bob!")]},
    config,
):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 86, 'total_tokens': 97}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-6fb0f9c4-4124-4240-a5dd-68fc4a4a4917-0', usage_metadata={'input_tokens': 86, 'output_tokens': 11, 'total_tokens': 97})]}}
---


In [18]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage("What's my name?")]},
    config,
):
    print(chunk)
    print("---")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you, Bob?', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 109, 'total_tokens': 123}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': 'fp_811936bd4f', 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {}}, id='run-b33fe8ae-82c9-41e0-b2c1-b6d88081118b-0', usage_metadata={'input_tokens': 109, 'output_tokens': 14, 'total_tokens': 123})]}}
---
